# nb_opma0_rayleigh

In [1]:
%matplotlib inline
import sys
sys.path.append('./opmap/opmap/')

In [2]:
from RawCam import RawCam
from VmemMap import VmemMap
from PhaseMap import PhaseMap
from PhaseVarianceMap import PhaseVarianceMap
from PhaseVariancePeakMap import PhaseVariancePeakMap

In [3]:
import copy
import pandas as pd
import os
import numpy as np

## Parameters

In [4]:
! find /mnt/recordings/ExperimentData/ -maxdepth 2 -type d -regex ".*20080902.*"
# ! ls /mnt/nas

/mnt/recordings/ExperimentData/20080902-1
/mnt/recordings/ExperimentData/20080902-1/C001S0007
/mnt/recordings/ExperimentData/20080902-1/C001S0035
/mnt/recordings/ExperimentData/20080902-1/C001S0006
/mnt/recordings/ExperimentData/20080902-2
/mnt/recordings/ExperimentData/20080902-2/C001S0027
/mnt/recordings/ExperimentData/20080902-2/C001S0026
/mnt/recordings/ExperimentData/20080902-2/C001S0024
/mnt/recordings/ExperimentData/20080902-2/first_position
/mnt/recordings/ExperimentData/20080902-2/C001S0025
/mnt/recordings/ExperimentData/20080902-3
/mnt/recordings/ExperimentData/20080902-3/C001S0023
/mnt/recordings/ExperimentData/20080902-3/C001S0026
/mnt/recordings/ExperimentData/20080902-3/C001S0024
/mnt/recordings/ExperimentData/20080902-3/C001S0035
/mnt/recordings/ExperimentData/20080902-3/C001S0025


In [41]:
srcdir = "/mnt/recordings/ExperimentData/"
outdir = "/mnt/nas/nb_opmap_rayleigh/"

if not os.path.exists(outdir) : os.makedirs(outdir)

_param = {
    'path' : None,
    'cam_type' : "sa4",
    'image_width' : None,
    'image_height' : None,
    'frame_start' : 0,
    'frame_end' : 1000
}

sessions = [
    "2004-08-06/C001S0004",
    "20080902-1/C001S0007"
]

arr_image_size = [
    256,
    512
]

sizes = np.arange(3, 12+1, 3)

thresholds = np.arange(0.4, 1.0, 0.1)


In [42]:
for s, sess in enumerate(sessions):
    
    param = copy.copy(_param)
    param["path"] = os.path.join(srcdir, sess)
    print(param["path"])

    ## RawCam
    param_opmap = copy.copy(_param)
    param_opmap['path'] = param['path']
    param_opmap['image_width'] = arr_image_size[s]
    param_opmap['image_height'] = arr_image_size[s]    
    cam = RawCam(**param_opmap)
    # cam.showFrame(showFrame)

    ## VmemMap
    vmem = VmemMap(cam)
    vmem.setDiffRange(diff_min=20)
    vmem.smooth(1+int(arr_image_size[s]/256)*4)
    # vmem.showFrame(showFrame)
    # vmem.showROI()

    ## PhaseMap
    pmap = PhaseMap(vmem)
    pmap.morphROI(closing=10)
    pmap.morphROI(erosion=10)
    pmap.setRectROI(top=10)
    # pmap.showFrame(showFrame)
    # pmap.showROI()
    
    ## Save
    dstdir = param['path'].replace(srcdir, outdir)
    print(dstdir)
    if not os.path.exists(dstdir): os.makedirs(dstdir)
    # vmem.saveMovie(os.path.join(dstdir, 'vmem.avi'))
    # pmap.saveImage(os.path.join(dstdir, 'pmap'))
    
    for size in sizes:
        
        ## PhaseVarianceMap
        pvmap = PhaseVarianceMap(pmap, size=size)
        pvmap.morphROI(erosion=size)
        # pvmap.showFrame(showFrame)
        # pvmap.showROI()

        ## Save
        dstdir_s = os.path.join(dstdir, 'pvmap_size_'+str(size))
        print(dstdir_s)
        if not os.path.exists(dstdir_s): os.makedirs(dstdir_s)
        pvmap.saveImage(os.path.join(dstdir_s, 'pvmap'))
        
        for threshold in thresholds:

            ## PhaseVariancePeakMap
            pvpmap = PhaseVariancePeakMap(pvmap, threshold=threshold)
            # pvpmap.showFrame(showFrame)
            # pvpmap.showROI()

            ## Save
            dstdir_t = os.path.join(dstdir_s, 'peak_threshold_'+str(threshold))
            print(dstdir_t)
            if not os.path.exists(dstdir_t): os.makedirs(dstdir_t)
            pvpmap.saveImage(os.path.join(dstdir_t, 'pvpmap'))

/mnt/recordings/ExperimentData/20080902-1/C001S0007


./opmap/opmap/VideoData.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.data = np.zeros((length, height, width), dtype=np.float32)
/root/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6/peak_threshold_0.4
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6/peak_threshold_0.5
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6/peak_threshold_0.6
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6/peak_threshold_0.7
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6/peak_threshold_0.8
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_6/peak_threshold_0.9
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_12
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_12/peak_threshold_0.4
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_12/peak_threshold_0.5
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_12/peak_threshold_0.6
/mnt/nas/nb_opmap_rayleigh/20080902-1/C001S0007/pvmap_size_12/peak_threshold_0.7
/mnt/nas/nb_opmap_rayleig

### generate dataframe

In [39]:
params = []
for s, sess in enumerate(sessions):    
    for size in sizes:
        for threshold in thresholds:
            
            param = copy.copy(_param)
            del(param["path"])
            param["session"] = sess
            param['image_width'] = arr_image_size[s]
            param['image_height'] = arr_image_size[s]    

            param["size"] = size
        
            param["threshold"] = threshold
            
            params.append(param)
            del(param)

dict_param = {}
for key in params[0].keys():
    dict_param[key] = [param[key] for param in params ]
    
dict_param
df = pd.DataFrame(dict_param)
df.to_csv(os.path.join(outdir, 'params.csv'))

df

,cam_type,frame_end,frame_start,image_height,image_width,session,size,threshold
0,sa4,1000,0,256,256,2004-08-06/C001S0004,3,0.4
1,sa4,1000,0,256,256,2004-08-06/C001S0004,3,0.5
2,sa4,1000,0,256,256,2004-08-06/C001S0004,3,0.6
3,sa4,1000,0,256,256,2004-08-06/C001S0004,3,0.7
4,sa4,1000,0,256,256,2004-08-06/C001S0004,3,0.8
5,sa4,1000,0,256,256,2004-08-06/C001S0004,3,0.9
6,sa4,1000,0,256,256,2004-08-06/C001S0004,6,0.4
7,sa4,1000,0,256,256,2004-08-06/C001S0004,6,0.5
8,sa4,1000,0,256,256,2004-08-06/C001S0004,6,0.6
9,sa4,1000,0,256,256,2004-08-06/C001S0004,6,0.7


df = pd.read_csv('./params.csv')
df